# Blackboard Grade Calculator

*Alexey Pronin*

*November 11, 2017*

## The brief description of the purpose of this notebook

The purpose of this notebook is to compute the current grades of the students listed in a gradebook file downloaded from [Blackboard](https://www.blackboard.com/index.html) in the coma-separated format. The output is generated in the form of a new CSV file which is obtained by ammending the original gradebook file with *Grades* and *Percentage* columns.

This notebook was created in the fall semester of 2017 when Blackboard experienced problems with the *Weighted Total* gradebook column -- the percentages displayed in the column were not computed correctly making it impossible to quickly check the current academic standing of students in my classes. The problem was fixed by the end of the semester but it motivated me to write this code just in case if something similar ever happens again.

## Preparing the data

### Entering the input data

The default name used in this notebook for the CSV file downloaded from Blackboard is *gradebook_file.csv*. A sample of the file is included in the notebook folder for demonstration purposes. The filename can be adjusted by changing the value of the `path` variable defined in the next cell. If the Blackboard file you are processesing is not located in the notebook folder then it is also going to be necessary to provide the complete path to the file.

In [1]:
(path <- "gradebook_file.csv")
#(path <- c(ENTER YOUR PATH HERE, COMMENT THE PRECEEDING LINE(S), AND UNCOMMENT THIS ONE))

[1] "gradebook_file.csv"

In order to compute the grades, it is also necessary to specify the categories of course assignments (e.g. homework, quizzes, tests, labs) that were utilized during your course and the weights (or percentages) for each of the categories. The default list of the categories with the corresponding percentages is defined below in a variable called `grading_rule`. To compute the grades correctly, the default list must be replaced with the one used for your course.

In [2]:
grading_rule <- data.frame('Category'   = c('HW', 'Quiz', 'Test 1', 'Test 2', 'Final', 'Absences'),
                           'Percentage' = c(  5,    20,      25,       25,       25,      0), 
                           stringsAsFactors = FALSE)
# COMMENT THE PRECEEDING THREE LINES
# UNCOMMENT THE THREE LINES FOLLOWING THIS ONE (DON'T FORGET TO ENTER YOUR CATEGORIES AND PERCENTAGES!)
#grading_rule <- data.frame('Category'   = c(ENTER YOUR CATEGORIES HERE),
#                           'Percentage' = c(ENTER YOUR PERCENTAGES HERE), 
#                           stringsAsFactors = FALSE)
grading_rule

Category,Percentage
HW,5
Quiz,20
Test 1,25
Test 2,25
Final,25
Absences,0


In [3]:
library(dplyr)

summarize(grading_rule, class(Percentage), class(Category))


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



class(Percentage),class(Category)
numeric,character


### The grade category and column names

* Later in this notebook, we will have to assign appropriate grade categories to the gradebook columns containig students' grades. This will be done with the help of regular expressions -- we will search the list of the gradebook column names for matches with the names of the grade categories. In order for this procedure to work smoothly, you will need to follow a few simple rules for naming of the grade categories and the gradebook columns. 

    * The category names should be capitalized to minimize the probabily of incorrent matches. Examples: *HW*, *Test 1*, *Quizzes*.
    
    * The names of the gradebook columns containig students's grades must begin with the names of the corresponding grade categories. For example, if you have two columns that belong to a single grade category *HW*, a good choice of the names for these columns would be *HW 1* and *HW 2*. Note that *HW1* and *HW2* or *HW_1* and *HW_2* should work just fine but not *Homework 1* and *Homework 2*. If you have a single column that belong to a single grade category, say, *Final*, some good choices of the name for the column are *Final Exam*, *Final_Exam*, or simply *Final*.
        
* The gradebook file will be stored in an R data frame. In R, the column names of data frames cannot include space characters -- in the process of reading the gradebook CSV file and converting it into a data frame, all space characters in the column names will be replaced by dots. To make it easier to work with these names in the future, it makes sense to do the exact same replacement in the grade category names stored in the *Category* column of the `grading_rule` data frame:

In [4]:
library(stringr)

(grading_rule <- mutate(grading_rule, Category = str_replace_all(str_trim(Category), " ", ".")))

Category,Percentage
HW,5
Quiz,20
Test.1,25
Test.2,25
Final,25
Absences,0


### The weights of the grade categories

Of course, the weights (or percentages) of all grade categories must add up to 100%. Let's do a quick check:

In [5]:
sum(grading_rule$Percentage)

[1] 100

## Reading the data

We will use the standard R `read.csv` function to read the gradebook file:

In [6]:
grades <- read.csv(path)

The gradebook file is now stored in a data frame called `grades`. Taking a peek at the first six rows:

In [7]:
head(grades)

Last.Name,First.Name,Username,Student.ID,Last.Access,Availability,Absences..Total.Pts..1.Score...52146,Weighted.Total..Total.Pts..up.to.11.15.Percentage...43356,Quiz.1..Total.Pts..10.Score...51772,Quiz.2..Total.Pts..15.Score...52030,HW.1..Total.Pts..3.Score...51289,HW.2..Total.Pts..6.Score...51470,HW.3..Total.Pts..7.Score...51771,HW.4..Total.Pts..7.Score...52031
Last_name_1,First_name_1,username_1,1,2019-02-14 17:57:00,Yes,NA,100.0,10,15,3,6,7,7
Last_name_2,First_name_2,username_2,2,2019-02-14 13:57:57,Yes,NA,100.0,10,15,3,6,7,7
Last_name_3,First_name_3,username_3,3,2019-02-18 16:48:58,Yes,NA,100.0,10,15,3,6,7,7
Last_name_4,First_name_4,username_4,4,2019-02-18 16:32:01,Yes,NA,96.8,9,15,3,6,7,7
Last_name_5,First_name_5,username_5,5,2019-02-18 15:06:58,Yes,NA,100.0,10,15,3,6,7,7
Last_name_6,First_name_6,username_6,6,2019-02-18 10:31:01,Yes,NA,90.4,7,15,3,6,7,7


We will store the column names of the data frame `grades` in the variable `col`:

In [8]:
col <- names(grades)
print(col)

 [1] "Last.Name"                                                
 [2] "First.Name"                                               
 [3] "Username"                                                 
 [4] "Student.ID"                                               
 [5] "Last.Access"                                              
 [6] "Availability"                                             
 [7] "Absences..Total.Pts..1.Score...52146"                     
 [8] "Weighted.Total..Total.Pts..up.to.11.15.Percentage...43356"
 [9] "Quiz.1..Total.Pts..10.Score...51772"                      
[10] "Quiz.2..Total.Pts..15.Score...52030"                      
[11] "HW.1..Total.Pts..3.Score...51289"                         
[12] "HW.2..Total.Pts..6.Score...51470"                         
[13] "HW.3..Total.Pts..7.Score...51771"                         
[14] "HW.4..Total.Pts..7.Score...52031"                         


## Processing the gradebook data

### Extracting maximum grades

In general, you should see that some of the columns contain students' grades for various types of assignments (tests, quizzes, homework, etc.); some other columns contain information about the students: their first and last names, usernames, student IDs, etc. In what follows, we will call the former *grade columns* and the latter *information columns*. Also, note that there is a columns containg information about students' weighted totals. We will treat it as a standalone column -- it is neither grade nor information column.

To compute the students' percentages, we will need to know the maximum grade the students could have received for each assignment. The maximum grades are embedded in the names of the grade columns -- note that each name of such a column contains a number following the *..Total.Pts..* part of the name. This number represents the maximum grade that we will need to extract. The extraction can be done with the help of the regular expressions syntax, as follows:

In [9]:
(max <- str_replace(col, "^.*Total\\.Pts\\.\\.([:digit:]*).*", "\\1"))

[1] "Last.Name"    "First.Name"   "Username"     "Student.ID"   "Last.Access" 
 [6] "Availability" "1"            ""             "10"           "15"          
[11] "3"            "6"            "7"            "7"

Obviously, at this stage `max` contains the maximum grades of the grade columns, the names of the information columns, and an empty string extracted from the column of weighted totals. This needs to be cleaned up. To do that, we must indentify the grade columns first. Recall that the names of the grade columns must begin with the name of one of the grade category stored in the *Category* column of `grading_rule`. This observation and the `str_detect` function of the `stringr` package will help us to detect these columns.

In [10]:
(grades_columns <- lapply(grading_rule$Category, 
                         function(x){which(str_detect(names(grades), paste("^", x, sep='')))}))

[[1]]
[1] 11 12 13 14

[[2]]
[1]  9 10

[[3]]
integer(0)

[[4]]
integer(0)

[[5]]
integer(0)

[[6]]
[1] 7

In [11]:
(grades_columns <- sort(unlist(grades_columns)))

[1]  7  9 10 11 12 13 14

Now we can select the maximum grades of the grade columns from `max`:

In [12]:
(max <- max[grades_columns])

[1] "1"  "10" "15" "3"  "6"  "7"  "7"

### Identifying categories for the grade columns

Different assignments listed in the gradebook correspond to different grade categories. For instance, the columns named *HW_1*, *HW_2*, *HW_3*, etc. clearly belong to the HW grade category. It would be helpful to have a vector that would show the grade categories for all grade columns. Let's make such a vector. The first step is to make an axiluary list of vectors called `category` which looks like this:

In [13]:
(category <- lapply(grading_rule$Category, function(x){ifelse(str_detect(names(grades), 
                                                   paste("^", x, sep='')), x, "")}))

[[1]]
 [1] ""   ""   ""   ""   ""   ""   ""   ""   ""   ""   "HW" "HW" "HW" "HW"

[[2]]
 [1] ""     ""     ""     ""     ""     ""     ""     ""     "Quiz" "Quiz"
[11] ""     ""     ""     ""    

[[3]]
 [1] "" "" "" "" "" "" "" "" "" "" "" "" "" ""

[[4]]
 [1] "" "" "" "" "" "" "" "" "" "" "" "" "" ""

[[5]]
 [1] "" "" "" "" "" "" "" "" "" "" "" "" "" ""

[[6]]
 [1] ""         ""         ""         ""         ""         ""        
 [7] "Absences" ""         ""         ""         ""         ""        
[13] ""         ""

The elements of this list correspond to the grade categories stored in the *Category* column of `grading_rule`. Each element is a vector whose length is equal to the total number of the gradebook columns and whose non-empty entries show the positions of the gradebook columns belonging to the corresponding grade category. Some vectors are empty because there are no gradebook columns that belong to this particular grade category. 

Next, we will concatenate the components of these vectors corresponding to the same gradebook column: 

In [14]:
(category <- apply(as.data.frame(category), 1, paste, collapse = ""))

[1] ""         ""         ""         ""         ""         ""        
 [7] "Absences" ""         "Quiz"     "Quiz"     "HW"       "HW"      
[13] "HW"       "HW"

We will use `category` later when we will be computing students' percentages. At this point, let's store the grade categories for the columns containing actual grades (as opposed to students' firt and last names, usernames, student ID, etc.) in a new variable called `category_grades`:

In [15]:
(category_grades <- category[grades_columns])

[1] "Absences" "Quiz"     "Quiz"     "HW"       "HW"       "HW"       "HW"

### Putting together information about the assignments, maximum grades and categories.

Finally, let put all information about the maxim grade and grade category for each of the grade column into a single data frame.

In [16]:
(info <- data.frame("Max" = max, 
                   "Category" = category_grades, 
                   "Assignment" = names(grades)[grades_columns], 
                   stringsAsFactors = FALSE))

Max,Category,Assignment
1,Absences,Absences..Total.Pts..1.Score...52146
10,Quiz,Quiz.1..Total.Pts..10.Score...51772
15,Quiz,Quiz.2..Total.Pts..15.Score...52030
3,HW,HW.1..Total.Pts..3.Score...51289
6,HW,HW.2..Total.Pts..6.Score...51470
7,HW,HW.3..Total.Pts..7.Score...51771
7,HW,HW.4..Total.Pts..7.Score...52031


A quick inspection of the `info` data frame printed above should convince us that the result makes sense.

### Computing the total maximum grade for each category

To compute students' percentages, we will need to know the total maximum grade for each grade category -- the sum of all maximum grades for all homework assignments, similar sum for all quizzes, then for all labs, etc. We will store the information about the total maximum grades in a new data frame `total_maximum`:  

In [17]:
(total_maximum <- info %>% 
                 group_by(Category) %>% 
                     summarize(Total_Max = sum(as.numeric(Max))))

Category,Total_Max
Absences,1
HW,23
Quiz,25


### Handling missing assingments

Any missing assignment grade would show up as an `NA` value in the gradebook. To compute the course grades accurately, it is necessary to replace all `NA`'s with zeroes:

In [18]:
grades[is.na(grades)] <- 0

## Computing students' percentages and grades

### Formula

Now we are ready to compute the percentages. To do that, we use the following formula: 

$$
\sum\limits_{\text{all categories}}\text{category weight}\times\frac{\text{total student's grade in this grade category}}{\text{maximum total grade for this grade category}}
$$ 

### Handling empty grade categories

Also, we need to take into account the possibility that if we do this computation in the middle of a semester, some of the categories might not be availalbe to us yet (e.g. *Final*).  To handle this situation, we first need to identify all empty grade categories and then adjust our computation of the percentages appropriently. Let's begin by indentifying the categories that are present in the gradebook.

In [19]:
(present_categories <- unique(info$Category))

[1] "Absences" "Quiz"     "HW"

The corresponding weights can be selected from the *Percentage* columns of `grading_rule`:

In [20]:
(mask <- grading_rule$Category %in% present_categories)

[1]  TRUE  TRUE FALSE FALSE FALSE  TRUE

In [21]:
(present_percentages <- grading_rule$Percentage[mask])

[1]  5 20  0

The total percentage is

In [22]:
(total_percentage <- sum(present_percentages))

[1] 25

To account for the fact that not all categories are present, we have to renormalize the computed percentage. This can be done by multiplying all computed percentages by an adjustment factor of `100/total_percentage`.

In [23]:
(adjustment_factor <- 100/total_percentage)

[1] 4

Of course, there are other ways to estimate the middle of the semester percentages -- here we are just following the procedure adopted by Blackboard.

### Computing the percentages

Now, let's compute the students' current percentages using the formula and the adjustment factor given above.

In [24]:
grades$Percentage <- 0

for (cat in present_categories){
    w <- grading_rule$Percentage[grading_rule$Category == cat]
    mask <- (category == cat)
    if (sum(mask) < 2) {
    result <- grades[ , mask]
    } else {
    result <- rowSums(grades[ , mask])
    }
    m <- total_maximum$Total_Max[(total_maximum$Category == cat)]
    grades$Percentage <- grades$Percentage + w*result/m
}

grades$Percentage <- adjustment_factor*grades$Percentage

### A custom-made rounding function

According to my syllabus, the computed percentages must be rounded to the nearest integer number. Normally, when we do rounding we follow the following simple rule: if the number you are rounding is followed by 5, 6, 7, 8, or 9, round the number up. Example: 66.5 rounded to the nearest integer is 67. If the number you are rounding is followed by 0, 1, 2, 3, or 4, round the number down. Example: 66.3 rounded to the nearest integer is 66. Unfortunately, the standard rounding function available in R (`round`) follows the so-called IEC 60559 standard (see also IEEE 754). According to this standard, when the number you are rounding is followed by 5 you are supposed to round it to the nearest even digit rather than up. Example:

In [25]:
round(84.5)
round(83.5)

[1] 84

[1] 84

I have chosen not to apply this standard in my classes. Instead, I prefere to follow the more traditional rounding rule described above and need to develop a custom rounding function for implementing this rule. I will call this function `round2` and here is how it is defined:

In [26]:
round2 = function(x, n = 0) {
    posneg = sign(x)
    z = abs(x)*10^n
    z = z + 0.5
    z = trunc(z)
    z = z/10^n
    z*posneg
}

And this is how it works:

In [27]:
round2(84.5)
round2(83.5)

[1] 85

[1] 84

which is exactly what I want.

### Computing the grades

Computing the final grade using the grading scheme from the syllabus:

In [28]:
for (i in 1:nrow(grades)){
        perc <- grades$Percentage[i]
        perc <- round2(perc)
        if ((perc >= 92)&(perc <= 100)){
            grades$Grade[i] <- "A"
        }
        if ((perc >= 89)&(perc < 92)){
            grades$Grade[i] <- "A-"
        }
        if ((perc >= 86)&(perc < 89)){
            grades$Grade[i] <- "B+"
        }
        if ((perc >= 82)&(perc < 86)){
            grades$Grade[i] <- "B"
        }
        if ((perc >= 79)&(perc < 82)){
            grades$Grade[i] <- "B-"
        }
        if ((perc >= 76)&(perc < 79)){
            grades$Grade[i] <- "C+"
        }
        if ((perc >= 72)&(perc < 76)){
            grades$Grade[i] <- "C"
        }
        if ((perc >= 69)&(perc < 72)){
            grades$Grade[i] <- "C-"
        }
        if ((perc >= 66)&(perc < 69)){
            grades$Grade[i] <- "D+"
        }
        if ((perc >= 62)&(perc < 66)){
            grades$Grade[i] <- "D"
        }
        if ((perc >= 59)&(perc < 62)){
            grades$Grade[i] <- "D-"
        }
        if ((perc >= 0)&(perc < 59)){
            grades$Grade[i] <- "F"
        }
    
    }

Two new columns *Percentage* and *Grade* has been added to `grades`:

In [29]:
print(names(grades))

 [1] "Last.Name"                                                
 [2] "First.Name"                                               
 [3] "Username"                                                 
 [4] "Student.ID"                                               
 [5] "Last.Access"                                              
 [6] "Availability"                                             
 [7] "Absences..Total.Pts..1.Score...52146"                     
 [8] "Weighted.Total..Total.Pts..up.to.11.15.Percentage...43356"
 [9] "Quiz.1..Total.Pts..10.Score...51772"                      
[10] "Quiz.2..Total.Pts..15.Score...52030"                      
[11] "HW.1..Total.Pts..3.Score...51289"                         
[12] "HW.2..Total.Pts..6.Score...51470"                         
[13] "HW.3..Total.Pts..7.Score...51771"                         
[14] "HW.4..Total.Pts..7.Score...52031"                         
[15] "Percentage"                                               
[16] "Grade"             

### Sanity check - compare it with Blackboard! 

To make sure that the computation was successful, let's compare the computed percentages with the students' weighted totals stored in the corresponding Blackboard column  (assuming that Blackboard is functioning properly). First, we will need to determine the index of the Blackboard column storing the weighted totals:

In [30]:
(weighted_total_idx <- which(str_detect(names(grades), '^Weighted.Total')))

[1] 8

Let's make sure that this is the right column:

In [31]:
(weighted_total_name <- names(grades)[weighted_total_idx])

[1] "Weighted.Total..Total.Pts..up.to.11.15.Percentage...43356"

Now, let's select only 5 columns of the gradebook and show them:

In [32]:
grades[c('Last.Name', 'First.Name', weighted_total_name, 'Percentage', 'Grade')]

Last.Name,First.Name,Weighted.Total..Total.Pts..up.to.11.15.Percentage...43356,Percentage,Grade
Last_name_1,First_name_1,100.0,100.0,A
Last_name_2,First_name_2,100.0,100.0,A
Last_name_3,First_name_3,100.0,100.0,A
Last_name_4,First_name_4,96.8,96.8,A
Last_name_5,First_name_5,100.0,100.0,A
Last_name_6,First_name_6,90.4,90.4,A-
Last_name_7,First_name_7,93.6,93.6,A
Last_name_8,First_name_8,96.8,96.8,A
Last_name_9,First_name_9,87.2,87.2,B+
Last_name_10,First_name_10,90.4,90.4,A-


Hopefully, the agreement is good! 

### Re-ordering the gradebook columns

Currently, the added columns, *Percentage* and *Grade*, appear at the very end of the `grades` data frame. We would like to place these columns next to the original column storing the weighted totals to make it easier to compare our result with the one computed by Blackboard (so that we won't have to go through a similar selection process in the future). To this end, we will create a new vector storing the column numbers exactly in the order we want them to be:

In [33]:
(col_number <- ncol(grades))

[1] 16

In [34]:
(new_order <- c(seq(1, weighted_total_idx), (col_number-1), col_number, 
                seq((weighted_total_idx + 1), (col_number-2))))

[1]  1  2  3  4  5  6  7  8 15 16  9 10 11 12 13 14

Reordered column namse of `grades` become:

In [35]:
print(names(grades[new_order]))

 [1] "Last.Name"                                                
 [2] "First.Name"                                               
 [3] "Username"                                                 
 [4] "Student.ID"                                               
 [5] "Last.Access"                                              
 [6] "Availability"                                             
 [7] "Absences..Total.Pts..1.Score...52146"                     
 [8] "Weighted.Total..Total.Pts..up.to.11.15.Percentage...43356"
 [9] "Percentage"                                               
[10] "Grade"                                                    
[11] "Quiz.1..Total.Pts..10.Score...51772"                      
[12] "Quiz.2..Total.Pts..15.Score...52030"                      
[13] "HW.1..Total.Pts..3.Score...51289"                         
[14] "HW.2..Total.Pts..6.Score...51470"                         
[15] "HW.3..Total.Pts..7.Score...51771"                         
[16] "HW.4..Total.Pts..7.

### Recording it all to a CSV file 

Finally, we will write the reordered `grades` data frame in a csv file which can then be uploaded back to Blackboard:

In [36]:
write.csv(grades[new_order], file = 'final_grades.csv')